In [49]:
'''
    First execute this code to get the images from the given pdf file
    It will save the image as the imcrement_num.png file
'''

import fitz
import io
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
file_name = 'Saleem HDFC 9660-1623914564274 (1) (2).pdf'
pdf_file = fitz.open(file_name)

for page_index in range(len(pdf_file)):
    page = pdf_file.loadPage(page_index)
    pix = page.getPixmap()
    image_name = f"image{page_index + 1}.png"
    pix.writePNG(image_name)
        

In [87]:
'''
    Now execute this code to detect the tables from the images generated earler
    Mark the table in the image and saving the file name as 'detectable_incement_num.jpg'
    
    Code commented at last is for extract the text from the detactable margin of image
    But because of some error it is currently not executable
'''

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import fitz

filename = 'Saleem HDFC 9660-1623914564274 (1) (2).pdf'
pdfile = fitz.open(filename)

def mark_region(image_path, n):
    img_read = cv2.imread(image_path)
    image_enhance = cv2.detailEnhance(img_read, sigma_s=8, sigma_r=0.15)
    img_bg_gray = cv2.cvtColor(image_enhance, cv2.COLOR_BGR2GRAY)
#     img_blur = cv2.GaussianBlur(img_bg_gray, (9,9), 0)
    img_thresh = cv2.adaptiveThreshold(img_bg_gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)
    img_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (6,6))
    img_dilate = cv2.dilate(img_thresh, img_kernel, iterations=4)
    img_median_blur = cv2.medianBlur(img_dilate, 9)
    img_cnts = cv2.findContours(img_dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(img_cnts) == 2:
        img_cnts = img_cnts[0]
    else:
        img_cnts = img_cnts[1]
        
    line_items_coordinates = []
    
    for a in img_cnts:
        area = cv2.contourArea(a)
        x,y,w,h = cv2.boundingRect(a)
        
        if y >= 600 and x <= 1000:
            if area > 10000:
                image = cv2.rectangle(img_read, (x,y), (550, y+h), color=(255,0,255), thickness=3)
                line_items_coordinates.append([(x,y), (550, y+h)])

        elif y >= 2400 and x<= 2000:
            image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
            line_items_coordinates.append([(x,y), (2200, y+h)])
            
    
    #plt.imshow(img_read)
    #cv2.namedWindow('detecttable', cv2.WINDOW_NORMAL)
    cv2.imwrite(f'detecttable{n + 1}.jpg',img_read)
    
    #return image, line_items_coordinates
    # load the original image
#     image = cv2.imread('detecttable1.jpg')

# #     # get co-ordinates to crop the image
# #     c = line_items_coordinates[0][1]

# #     # cropping image img = image[y0:y1, x0:x1]
# #     img = image[c[0][1]:c[1][1], c[0][0]:c[1][0]]    

# #     plt.figure(figsize=(10,10))
# #     plt.imshow(img)

# #     # convert the image to black and white for better OCR
#     ret,thresh1 = cv2.threshold(image,120,255,cv2.THRESH_BINARY)

#     # pytesseract image to string to get results
#     text = str(pytesseract.image_to_string(thresh1, config='--psm 6'))
#     print(text)
    
    

for i in range(len(pdfile)):
    img_name = f"image{i + 1}.png"
    mark_region(img_name, i)